In [1]:
import pandas as pd
import numpy as np
import re
import time
import string
string.punctuation
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
nltk.download("punkt")
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download("wordnet")
nltk.download("omw-1.4")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
news_df = pd.read_pickle('News_df_scored.pkl')

In [3]:
news_df.head(2)

,text,real,subject,title,date,id,fe_score,fk_score,cl_score,ari_score,...,std_score,syllable_count,word_count,sentence_count,char_count,letter_count,polysyllab_count,monosyllab_count,clean_text,std_score_num
0,House Dem Aide: We Didn’t Even See Comey’s Let...,0,None,None,None,21508,57.61,10.7,11.02,13.3,...,11th and 12th grade,1249,815,37,4101,3980,96,516,house dem aide we didn t even see comey s let...,11.5
1,"Why the Truth Might Get You Fired October 29, ...",0,None,None,None,21509,51.68,10.9,11.49,12.5,...,10th and 11th grade,1938,1246,64,6405,6174,192,826,why the truth might get you fired october ...,10.5


In [6]:
cl_low = news_df[news_df['cl_score'] < 0]
cl_low

,text,real,subject,title,date,id,fe_score,fk_score,cl_score,ari_score,...,std_score,syllable_count,word_count,sentence_count,char_count,letter_count,polysyllab_count,monosyllab_count,clean_text,std_score_num
609,Zenit 2 2 0 0 9 3 6 \nDundalk 2 1 1 0 2 1 4 \n...,0,None,None,None,22509,19.34,31.6,-2.77,31.8,...,31st and 32nd grade,296,254,3,586,584,8,220,zenit dundalk al...,31.5
19641,Although the Nov. 8 U.S. presidential election...,1,None,None,None,56789,92.49,1.4,-1.16,0.6,...,0th and 1st grade,430,329,77,1388,1218,23,264,although the nov u s presidential election...,0.5
53905,WASHINGTON (Reuters) - Although the Nov. 8 U.S...,1,politicsNews,Factbox: Early voting starts next month in U.S...,"August 29, 2016",92385,92.49,1.4,-0.98,0.7,...,0th and 1st grade,434,331,77,1408,1235,24,265,washington reuters although the nov u s...,0.5


In [7]:
'still fuzzy duplicates, maybe I can leverage the readability scores further. Also need to remove item at 609'

'still fuzzy duplicates, maybe I can leverage the readability scores further. Also need to remove item at 609'

In [8]:
news_df['text'][19641][:350]

'Although the Nov. 8 U.S. presidential election is more than two months away, the first ballots will be cast in just a few weeks when several states begin early voting. Thirty-eight of the 50 U.S. states have some form of early voting in this year’s race between Democrat Hillary Clinton and Republican Donald Trump. Pushing supporters to vote early h'

In [10]:
news_df['text'][53905][:350]

'WASHINGTON (Reuters) - Although the Nov. 8 U.S. presidential election is more than two months away, the first ballots will be cast in just a few weeks when several states begin early voting. Thirty-eight of the 50 U.S. states have some form of early voting in this year’s race between Democrat Hillary Clinton and Republican Donald Trump. Pushing sup'

In [11]:
column_names = [
    'fe_score', 
    'fk_score',
    'cl_score',
    'ari_score',
    'dc_score',
    'dw_score',
    'ls_score',
    'gf_score',
    'std_score',
    'syllable_count',
    'word_count',
    'sentence_count',
    'char_count',
    'letter_count',
    'polysyllab_count',
    'monosyllab_count'
    ]

In [13]:
news_df.drop(609, inplace = True)
news_df.reset_index(drop = True, inplace = True)


In [14]:
news_df = news_df.sort_values(column_names)
news_df.head(50)

,text,real,subject,title,date,id,fe_score,fk_score,cl_score,ari_score,...,std_score,syllable_count,word_count,sentence_count,char_count,letter_count,polysyllab_count,monosyllab_count,clean_text,std_score_num
82913,21st Century Wire says The investigation team ...,0,Middle-east,Extreme Left #DISRUPTJ20 Plot to ‘Acid Bomb’ I...,"January 17, 2017",128501,0.09,28.7,14.17,34.7,...,34th and 35th grade,205,124,2,662,654,25,76,st century wire says the investigation team ...,34.5
9753,Historical documents newly released by Russia’...,0,None,None,None,35511,0.09,28.7,14.87,35.5,...,14th and 15th grade,105,62,1,341,334,14,35,historical documents newly released by russia ...,14.5
11350,Sputnik Estonia employees received letters fro...,0,None,None,None,45252,0.42,24.4,18.18,29.3,...,12th and 13th grade,84,45,1,270,268,11,23,sputnik estonia employees received letters fro...,12.5
82510,Wow! What a list of accomplishments! The probl...,0,left-news,THE LIST OF OBAMA’S HISTORIC FIRSTS AKA HOW CH...,"Apr 14, 2015",127312,0.59,28.5,14.87,35.6,...,28th and 29th grade,3123,1846,30,10298,9955,384,1090,wow what a list of accomplishments the probl...,28.5
10900,The UNESCO’s revised edition of the Internatio...,0,None,None,None,42310,0.76,20.1,20.43,23.8,...,19th and 20th grade,118,56,2,371,361,17,26,the unesco s revised edition of the internatio...,19.5
3215,A stunning Foreign Intelligence Service (SVR) ...,0,None,None,None,26210,0.80,28.4,15.16,36.1,...,28th and 29th grade,1429,858,14,4900,4669,147,507,a stunning foreign intelligence service svr ...,28.5
8164,A very sobering Ministry of Defense ( MoD ) re...,0,None,None,None,33083,1.14,30.3,14.53,38.7,...,22nd and 23rd grade,1247,762,11,4127,4024,125,456,a very sobering ministry of defense mod re...,22.5
1990,We Are Change \nIn this video Luke Rudkowski c...,0,None,None,None,24481,1.19,15.8,39.36,38.5,...,13th and 14th grade,381,163,15,1884,1625,49,86,we are change in this video luke rudkowski co...,13.5
5611,Cerrar Normas comunitarias \nEl registro y la ...,0,None,None,None,29574,1.37,26.1,15.39,31.3,...,23rd and 24th grade,1050,576,11,3240,3157,141,292,cerrar normas comunitarias el registro y la a...,23.5
7436,The Debate Washington's Syria policy \nIn this...,0,None,None,None,32067,1.78,32.1,12.56,41.0,...,12th and 13th grade,119,77,1,392,380,11,48,the debate washington s syria policy in this ...,12.5


In [15]:
'Also have russian and arabic?, I only want English language. I will start here'

'Also have russian and arabic?, I only want English language. I will start here'

In [16]:
def has_cyrillic(text):
    return bool(re.search('[а-яА-Я]', text))

In [17]:
news_df.reset_index(drop = True, inplace = True)


In [19]:
x = len(news_df)
r_list = []
for i in range(x):
    if has_cyrillic(news_df['text'][i]) == True:
        r_list.append(i)

In [21]:
x1 = len(r_list)


In [23]:
r_df = news_df.filter(items=(r_list), axis=0)

In [26]:
r_df.reset_index(drop = True, inplace = True)


In [27]:
'''for i in range(x1):
    print(R_df['id'][i]) 
    print(R_df['text'][i])'''

"for i in range(x1):\n    print(R_df['id'][i]) \n    print(R_df['text'][i])"

In [28]:
'''
I ran the code commented out above to review the articles. In the end I decided to drop all of them as most where not English.
'''

'\nI ran the code commented out above to review the articles. In the end I decided to drop all of them as most where not English.\n'

In [29]:
news_df.drop(r_list, axis = 0, inplace = True)
news_df.reset_index(drop =True, inplace = True)

In [30]:
def has_arabic(text):
    return bool(re.search('^[\u0621-\u064A]', text))

In [31]:
x = len(news_df)
arabic = []
for i in range(x):
    if has_arabic(news_df['text'][i]) == True:
        arabic.append(i)

In [32]:
'''for i in arabic:
    print(News_df['text'][i])
    '''

"for i in arabic:\n    print(News_df['text'][i])\n    "

In [35]:
# Ran the above code and confirmed that these were arabic

In [34]:
news_df.drop(arabic, axis = 0, inplace = True)
news_df.reset_index(drop =True, inplace = True)

In [36]:
'''okay for the hard part, using the readability scores as fuzzy matching, values were chosen by trial and error.
Method assumes sorted values and is reasonably fast.
'''

'okay for the hard part, using the readability scores as fuzzy matching, values were chosen by trial and error.\nMethod assumes sorted values and is reasonably fast.\n'

In [33]:
'''These values and method was developed by trial and error and is catching fuzzy matches via readability scores.
This was not a 100% reliable method there were a very small number of false positives that were eliminated. However, the 
benefit was judged to be warranted'''

'These values and method was developed by trial and error and is catching fuzzy matches via readability scores'

In [38]:
start_time = time.time()
x = len(news_df)
dup = []
dup1 = []
for i in range(x-1):
        check = abs(news_df['fe_score'][i] - news_df['fe_score'][i+1])
        check1 = abs(news_df['word_count'][i] - news_df['word_count'][i+1])
        check2 = abs(news_df['std_score_num'][i] - news_df['std_score_num'][i+1])
        check3 = abs(news_df['cl_score'][i] - news_df['cl_score'][i+1])
        check4 = abs(news_df['dw_score'][i] - news_df['dw_score'][i+1])
        check5 = abs(news_df['gf_score'][i] - news_df['gf_score'][i+1])
        check6 = abs(news_df['dc_score'][i] - news_df['dc_score'][i+1])
        if check <.01 and check1 < 15 and check2 < 1 and check3 <.3 and check4 <3 and check5 <.5 and check6 <.2: 
            dup.append(i)
            dup1.append(i+1)
        
print("--- %s seconds ---" % (time.time() - start_time))

--- 14.95809030532837 seconds ---


In [39]:
#x = len(dup)
dup_f= dup +dup1
dup_f.sort()


In [40]:
'''pd.options.display.max_colwidth = 500
x = len(dup_f)
for i in range(x):
    print(News_df['text'][dup_f[i:20]])'''
        

"pd.options.display.max_colwidth = 500\nx = len(dup_f)\nfor i in range(x):\n    print(News_df['text'][dup_f[i:20]])"

In [42]:
'The commented out code above were almost all fuzzy duplicates. I will drop one set.'

'The commented out code above were almost all fuzzy duplicates. I will drop one set.'

In [44]:
news_df.drop(dup, inplace = True)

In [45]:
news_df.reset_index(drop = True, inplace = True)

In [46]:
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

In [48]:
start_time = time.time()
news_df['clean_text']= news_df['clean_text'].apply(lambda x:remove_punctuation(x))
news_df.head()
print("--- %s seconds ---" % (time.time() - start_time))

--- 46.51270771026611 seconds ---


In [49]:
news_df['clean_text'][:2]

0      st century wire says the investigation team ...
1    historical documents newly released by russia ...
Name: clean_text, dtype: object

In [50]:
wnl = WordNetLemmatizer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()

In [51]:
def lemmatize_text(text):
    return [wnl.lemmatize(w) for w in w_tokenizer.tokenize(text)]



In [52]:
start_time = time.time()
news_df['clean_text'] = news_df.clean_text.apply(lemmatize_text)
print("--- %s seconds ---" % (time.time() - start_time))

--- 328.9231264591217 seconds ---


In [53]:
news_df['clean_text'] = [' '.join(map(str, l)) for l in news_df['clean_text']]


In [54]:
news_df['clean_text'].apply(lambda x: x.lower())

0        st century wire say the investigation team at ...
1        historical document newly released by russia s...
2        sputnik estonia employee received letter from ...
3        wow what a list of accomplishment the problem ...
4        the unesco s revised edition of the internatio...
                               ...                        
82649    ukraine would sell even it nuclear bomb if the...
82650    monday on abc s the view joy behar said presid...
82651    i actually think you guy have done good work h...
82652    i will be a camera and would like to know if t...
82653    vote for trump we can do it get out and vote a...
Name: clean_text, Length: 82654, dtype: object

In [55]:
vectorizer = TfidfVectorizer(max_features = 500, stop_words = 'english')

In [56]:
X_train, X_test, y_train, y_test = train_test_split(news_df.drop(columns=['real', 'title', 'date', 'text', 'subject','std_score']), 
                                                    news_df.real, test_size=0.25, stratify = news_df.real,
                                                    random_state=1701)

In [57]:
X_train.reset_index(drop = True, inplace = True)
X_train.head(2)

,id,fe_score,fk_score,cl_score,ari_score,dc_score,dw_score,ls_score,gf_score,syllable_count,word_count,sentence_count,char_count,letter_count,polysyllab_count,monosyllab_count,clean_text,std_score_num
0,78511,65.73,7.6,9.10,8.5,9.41,166,7.375000,8.91,1140,786,56,3831,3656,83,542,want to get this briefing by email here s the ...,8.5
1,76190,48.27,14.3,12.37,18.6,9.88,113,21.666667,15.97,869,561,18,2914,2813,75,363,democratic unionist party leader arlene foster...,14.5


In [58]:
y_train.reset_index(drop = True, inplace = True)
y_train.head(2)

0    1
1    1
Name: real, dtype: int64

In [59]:
X_test.reset_index(drop = True, inplace = True)
X_test.head(2)

,id,fe_score,fk_score,cl_score,ari_score,dc_score,dw_score,ls_score,gf_score,syllable_count,word_count,sentence_count,char_count,letter_count,polysyllab_count,monosyllab_count,clean_text,std_score_num
0,71266,69.11,8.3,9.28,10.5,8.55,89,8.166667,10.38,759,533,28,2532,2440,60,380,donald j trump took credit on wednesday for a ...,8.5
1,77753,63.73,10.4,10.33,13.6,8.61,213,13.750000,12.83,1908,1338,55,6505,6308,137,946,the summer olympics in rio de janeiro may end ...,13.5


In [60]:
y_test.reset_index(drop = True, inplace = True)
y_test.head(2)

0    1
1    1
Name: real, dtype: int64

In [61]:
'Code adapted from Data Camp slides'

'Code adapted from Data Camp slides'

In [62]:
start_time = time.time()
news_train_trans = vectorizer.fit_transform(X_train['clean_text'])
print("--- %s seconds ---" % (time.time() - start_time))

--- 36.42403483390808 seconds ---


In [63]:
news_test_trans = vectorizer.transform(X_test['clean_text'])

In [65]:
news_test_df = pd.DataFrame(news_test_trans.toarray(), 
                          columns=vectorizer.get_feature_names_out()).add_prefix('TFIDF_')
news_test_df.head(2)

,TFIDF_able,TFIDF_access,TFIDF_according,TFIDF_account,TFIDF_accused,TFIDF_act,TFIDF_action,TFIDF_actually,TFIDF_added,TFIDF_administration,...,TFIDF_won,TFIDF_word,TFIDF_work,TFIDF_worker,TFIDF_working,TFIDF_world,TFIDF_wrote,TFIDF_year,TFIDF_york,TFIDF_young
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04306,0.079719,...,0.000000,0.0,0.037984,0.0,0.0,0.000000,0.0,0.050592,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03044,0.000000,...,0.064104,0.0,0.000000,0.0,0.0,0.343723,0.0,0.000000,0.0,0.0


In [66]:
news_train_df = pd.DataFrame(news_train_trans.toarray(), 
                          columns=vectorizer.get_feature_names_out()).add_prefix('TFIDF_')
news_train_df.head(2)

,TFIDF_able,TFIDF_access,TFIDF_according,TFIDF_account,TFIDF_accused,TFIDF_act,TFIDF_action,TFIDF_actually,TFIDF_added,TFIDF_administration,...,TFIDF_won,TFIDF_word,TFIDF_work,TFIDF_worker,TFIDF_working,TFIDF_world,TFIDF_wrote,TFIDF_year,TFIDF_york,TFIDF_young
0,0.000000,0.0,0.0,0.0,0.073841,0.0,0.0,0.0,0.06195,0.0,...,0.000000,0.000000,0.054647,0.0,0.000000,0.053810,0.0,0.036393,0.061541,0.0
1,0.073135,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,...,0.067625,0.072737,0.113305,0.0,0.068093,0.055785,0.0,0.000000,0.000000,0.0


In [67]:
examine_row = news_train_df.iloc[0]

In [68]:
examine_row.sort_values(ascending = False)

TFIDF_woman     0.313603
TFIDF_mr        0.271877
TFIDF_job       0.255753
TFIDF_want      0.204045
TFIDF_group     0.201539
                  ...   
TFIDF_help      0.000000
TFIDF_held      0.000000
TFIDF_health    0.000000
TFIDF_head      0.000000
TFIDF_young     0.000000
Name: 0, Length: 500, dtype: float64

In [69]:
news_train_df = pd.concat([X_train, news_train_df], axis = 1, sort = False)


In [70]:
news_test_df = pd.concat([X_test, news_test_df], axis = 1, sort = False)


In [71]:
news_train_df.to_pickle('News_train_df.pkl')

In [72]:
news_test_df.to_pickle('News_test_df.pkl')

In [73]:
y_train.to_pickle('y_train.pkl')

In [74]:
y_test.to_pickle('y_test.pkl')